**Inputs**
* ImageCollection(s) containing hazard probabilities for 2030 and 2065

**Outputs**
* Images: two-band (alpha, beta), one pair per magnitude

In [29]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWj_H3PkzACBllyHJRnJu6v4Uhh9s1rs2MLJhlBMHBh2erlkiB94WUA

Successfully saved authorization token.


In [30]:
ee.Initialize()

In [26]:
WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
MODELS = ['ACCESS1-0', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM', 'MIROC-ESM-CHEM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M']
hazardnames = ['dryspells']
hazardname = 'dryspells'
HIST_START = 1980
HIST_END = 2019
num_bins = 365
bin_width = 1

In [5]:
era_mags = {}
for h in hazardnames:
    era_mags[h] = ee.ImageCollection('users/tedwongwri/dataportal/hist-magnitudes/' + h)

In [13]:
def alpha_beta(magnitude):
# alpha and beta parameters for MAP prob formula
# alpha: historical "successes" plus 1
# beta: historical "failures" plus 1
    successes = ee.Image.constant(0).rename('alpha')
    failures = ee.Image.constant(0).rename('beta')
    mag_img = ee.Image.constant(magnitude)
    for hist_year in range(HIST_START, HIST_END + 1):
        # for pixel == magnitude
        histyear = era_mags[hazardname].filterMetadata('year', 'equals', hist_year).first()
        successes = successes.add(histyear.round().eq(mag_img).multiply(1))
        failures = failures.add(histyear.round().eq(mag_img).multiply(-1).add(1))
        # for pixel gte magnitude
        #successes = successes.add(ehe_histyear.gte(magnitude).multiply(ehe_histyear.lt(magnitude + 5)))
        #failures = failures.add(ehe_histyear.lt(magnitude).add(ehe_histyear.gte(magnitude + 5)).min(1))
    alpha = successes.add(1).set('source', 'era5').set('magnitude', magnitude)
    beta = failures.add(1).set('source', 'era5').set('magnitude', magnitude)
    ic = ee.ImageCollection([alpha, beta])
    twoband_image = ic.toBands().rename(['alpha', 'beta'])
    return twoband_image

In [33]:
for bin_idx in range(num_bins):
    magnitude = (bin_idx + 1) * bin_width
    img = alpha_beta(magnitude).set('magnitude', magnitude)
    task = ee.batch.Export.image.toAsset(**{
      'image': img,
      'description': 'alphabeta_{0}_{1}'.format(hazardname, str(magnitude)),
      'assetId': 'users/tedwongwri/dataportal/alphabeta/{0}/magnitude_{1}'.format(hazardname, str(magnitude)),
      'region': WHOLE_GLOBE,
      'crs': 'EPSG:4326',
      'dimensions': '1440x720'
    })
    task.start()